In [6]:
import pandas as pd
import numpy as np

file_path = "/content/hospital_inpatient_discharges_totalhipreplacement.csv"
df = pd.read_csv(file_path)

df.head()


,health_service_area,hospital_county,operating_certificate_number,facility_id,facility_name,age_group,zip_code_3_digits,gender,race,ethnicity,...,apr_mdc_code,apr_mdc_description,apr_severity_of_illness_code,apr_severity_of_illness_description,apr_risk_of_mortality,apr_medical_surgical_description,attending_provider_license_number,operating_provider_license_number,total_charges,total_costs
0,Western NY,Allegany,228000,39,Memorial Hosp of Wm F & Gertrude F Jones A/K/A...,70 or Older,148,F,White,Not Span/Hispanic,...,8,Diseases and Disorders of the Musculoskeletal ...,2,Moderate,Minor,Surgical,213053,213053,35681.75,17400.79
1,Western NY,Allegany,228000,39,Memorial Hosp of Wm F & Gertrude F Jones A/K/A...,50 to 69,148,F,White,Not Span/Hispanic,...,8,Diseases and Disorders of the Musculoskeletal ...,2,Moderate,Minor,Surgical,213053,213053,34289.25,16657.95
2,Western NY,Allegany,228000,39,Memorial Hosp of Wm F & Gertrude F Jones A/K/A...,50 to 69,147,M,White,Not Span/Hispanic,...,8,Diseases and Disorders of the Musculoskeletal ...,2,Moderate,Minor,Surgical,213053,213053,30436.00,14703.62
3,Western NY,Allegany,228000,39,Memorial Hosp of Wm F & Gertrude F Jones A/K/A...,50 to 69,147,M,White,Not Span/Hispanic,...,8,Diseases and Disorders of the Musculoskeletal ...,1,Minor,Minor,Surgical,213053,213053,28699.00,13903.30
4,Western NY,Allegany,228000,39,Memorial Hosp of Wm F & Gertrude F Jones A/K/A...,50 to 69,148,M,White,Not Span/Hispanic,...,8,Diseases and Disorders of the Musculoskeletal ...,1,Minor,Minor,Surgical,213053,213053,18421.00,8258.43


In [7]:
df.info()
df.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26594 entries, 0 to 26593
Data columns (total 30 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   health_service_area                  26594 non-null  object 
 1   hospital_county                      26594 non-null  object 
 2   operating_certificate_number         26594 non-null  int64  
 3   facility_id                          26594 non-null  int64  
 4   facility_name                        26594 non-null  object 
 5   age_group                            26594 non-null  object 
 6   zip_code_3_digits                    26574 non-null  object 
 7   gender                               26594 non-null  object 
 8   race                                 26594 non-null  object 
 9   ethnicity                            26594 non-null  object 
 10  length_of_stay                       26594 non-null  int64  
 11  type_of_admission           

,0
health_service_area,0
hospital_county,0
operating_certificate_number,0
facility_id,0
facility_name,0
age_group,0
zip_code_3_digits,20
gender,0
race,0
ethnicity,0


In [8]:
num_cols = df.select_dtypes(include=["int64", "float64"]).columns

for col in num_cols:
    df[col] = df[col].fillna(df[col].median())


In [9]:
cat_cols = df.select_dtypes(include=["object"]).columns

for col in cat_cols:
    df[col] = df[col].fillna(df[col].mode()[0])


In [10]:
for col in cat_cols:
    df[col] = (
        df[col]
        .str.strip()
        .str.lower()
    )


In [11]:
df.columns


Index(['health_service_area', 'hospital_county',
       'operating_certificate_number', 'facility_id', 'facility_name',
       'age_group', 'zip_code_3_digits', 'gender', 'race', 'ethnicity',
       'length_of_stay', 'type_of_admission', 'patient_disposition',
       'discharge_year', 'ccs_diagnosis_code', 'ccs_diagnosis_description',
       'ccs_procedure_code', 'ccs_procedure_description', 'apr_drg_code',
       'apr_drg_description', 'apr_mdc_code', 'apr_mdc_description',
       'apr_severity_of_illness_code', 'apr_severity_of_illness_description',
       'apr_risk_of_mortality', 'apr_medical_surgical_description',
       'attending_provider_license_number',
       'operating_provider_license_number', 'total_charges', 'total_costs'],
      dtype='object')

In [12]:
def remove_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return data[(data[column] >= lower) & (data[column] <= upper)]


In [14]:
df = remove_outliers_iqr(df, "total_charges")
df = remove_outliers_iqr(df, "total_costs")
df = remove_outliers_iqr(df, "length_of_stay")

In [18]:
df["charge_to_cost_ratio"] = df["total_charges"] / df["total_costs"]

In [21]:
df["charge_to_cost_ratio"] = df["charge_to_cost_ratio"].replace([np.inf, -np.inf], np.nan)
df["charge_to_cost_ratio"] = df["charge_to_cost_ratio"].fillna(df["charge_to_cost_ratio"].median())

In [22]:
df["efficiency_index"] = 1 / (df["total_costs"] * df["length_of_stay"])

In [23]:
df.describe()
df.isnull().sum()


,0
health_service_area,0
hospital_county,0
operating_certificate_number,0
facility_id,0
facility_name,0
age_group,0
zip_code_3_digits,0
gender,0
race,0
ethnicity,0


In [24]:
df.to_csv("cleaned_hip_replacement_data.csv", index=False)
